# Notas do curso Séries Temporais com Python

## Capítulo 5: Machine Learning em Séries Temporais

### 1. Seleção de variáveis com random forest

Uma parte importante da análise de dados é escolher corretamente o grupo de variáveis e a quantidade de dados a ser utilizada a fim de obter a melhor performance em um modelo de previsão. Nesta seção iremos abordar o uso da técnica de Random Forest com o objetivo de escolher quais são os períodos mais relevantes para melhorar a performance da previsão de períodos futuros.

In [ ]:
# Biblioteca de leitura de dados
import pandas as pd

# Importando dataset (com a correta leitura da data)
ts = pd.read_csv( "../Material/Dados/AirPassengers.csv", header=0, parse_dates=[0], index_col=0, squeeze=True)

In [ ]:
# Remover o efeito sazonal derivando a série temporal
dts = ts.diff(12) # NA´s entre índices 1 e 12

# Remover os 12 primeiros meses
dts = dts[12:]

# Gráfico e head da série derivada
print(dts.head(13)); dts.plot()

In [ ]:
# Vamos criar uma série temporal com 12 meses para servir de base para o Random Forest
# O objetivo é fazer combinações dos períodos passados e permitir o algoritmo escolher a melhor entre elas
base = pd.DataFrame()

for i in range(12, 0, -1):
    # Algo de errado não está certo!
    base['t-'+str(i)] = dts.shift(i)
    base['t'] = dts.values

print(base.head(13))
base = base[13:] # removendo NaN's

In [ ]:
# Algoritmo de regressão usando random forest
from sklearn.ensemble import RandomForestRegressor

# Formatando dados para usar no regressor
array = base.values
X = array[:,0:-1]
Y = array[:,-1]

# Criando modelo
mod = RandomForestRegressor(n_estimators = 500, random_state = 1)

# Treinando modelo
mod.fit(X,Y)

# Impressão de resultados
print(mod.feature_importances_)

# Identificar qual lag apresenta maio importância (maior valor do feature_importances) ----------------------------
names  = base.columns.values[0:-1] # obtendo os nomes 't-n' das colunas
ticks = [i for i in range(len(names))] # definindo índices para cada nome

# Importando biblioteca de gráficos
from matplotlib import pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

# Criando gráficos de barras a partir do feature_importances_
plt.bar(ticks, mod.feature_importances_)
plt.xticks(ticks, names)
plt.show()

In [ ]:
# Importando biblioteca para seleção de atributos usando RFE
from sklearn.feature_selection import RFE

# Criando regressor
rfe = RFE(RandomForestRegressor(n_estimators = 500, random_state = 1), 4)

# Treinando regressor
fit = rfe.fit(X,Y)

# Laço de impressão para observar resultados
for i in range(len(fit.support_)):
    if (fit.support_[i]):
        print(names[i])

A partir desses resultados (gráfico e laço de impressão) é possível observar que os períodos mais importantes para a performance da previsão são os 4 períodos anteriores

### 2. Prophet

Esse é um algoritmo de previsão de séries temporais criado pelo Facebook

In [1]:
# Importando biblioteca para leitura de dados
import pandas as pd

# Importando bilioteca prophet
from fbprophet import Prophet

In [2]:
# Importando dados
data = pd.read_csv("../Material/Dados/train_1.csv").T 

# Definindo parâmetros relevantes
N = 180 # 60 # Número de dias para teste
i = 1800 # Série temporal para treino

In [3]:
# Separando dados de treino e teste
treino, teste = data.iloc[0:-N,:], data.iloc[-N:,:]

# Tratando valores NA
teste_noNA = teste.T.fillna(method = 'ffill').T
treino_noNA = treino.T.iloc[:,1:].fillna(method = 'ffill').T

# Importando biblioteca para cálculos numéricos
import numpy as np

# Removendo outliers: todos os dados que estão fora de 5 desvios padrão da média móvel de 50 dias
base = treino_noNA.iloc[:,i].to_frame()
base.columns = ['visits']
base['movavg'] = pd.Series(base.visits).rolling(window = 50).mean()
std_mult = 1.5 # padrão do desvio-padrão a procurar
base.loc[np.abs(base.visits-base.visits.mean())>=(std_mult*base.visits.std()),'visits'] = base.loc[np.abs(base.visits-base.visits.mean())>=(std_mult*base.visits.std()),'movavg']
    
base.index = pd.to_datetime(base.index)

In [4]:
# Mudança nos labels
X = pd.DataFrame(index = range(len(base)))
X['ds'] = base.index
X['y'] = base['visits'].values
print(X.head())

# Salvando dados
import pickle
pickle.dump(X, open("prophet_train_data", "wb" ))

          ds     y
0 2015-07-01   1.0
1 2015-07-02   6.0
2 2015-07-03   8.0
3 2015-07-04   7.0
4 2015-07-05  12.0


In [ ]:
# Procurando por NA´s
print(np.unique(np.isnan(X.y)))
print(np.unique(np.isnan(X.ds)))

# Removendo NA´s
X.dropna(inplace = True)

# Primeiras linhas do dataframe resultante
X.head()

In [ ]:
# Criando modelo do Prophet
mod = Prophet(yearly_seasonality = True)

# Treinando modelo
mod.fit(X, algorithm = 'Newton') # mesmo com a mudança de algoritmo, o kernel morre ao tentar executar esta linha
# Treinei o modelo no google colab
# No google colab também não funcionou

# Salvando modelo treinado
pickle.dump(mod, open("prophet_model", "wb" ))

In [ ]:
# Lendo o modelo
mod = pickle.load(open("prophet_model", "rb" ) 

# Prevendo a partir do modelo treinado
future = mod.make_future_dataframe(periods = N)

# Últimas linhas do dataframe de previsão
future.tail()

In [ ]:
# Prevendo as próximas 60 observações
fcast = mod.predict(future)
fcast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

# Gráfico da previsão
mod.plot(fcast)